In [25]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

import tensorflow.keras as keras
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [19]:
import pandas as pd
import warnings


warnings.filterwarnings('ignore')

class Train_Data():
    
    '''
    Train_Data class Loads the data from multiple Training JSON files in Pandas Dataframes
    '''
   
    def __init__(self, path , filenames):
        '''
            Parameters
            ------------
            path: directory where English to SQL translation JSON are placed
            filenames: Json file names containing Train data
        '''
        self.path = path
        self.filenames = filenames
        self.df_train = pd.DataFrame()
        for f in self.filenames:
            print("Reading file at path", self.path+f)
            try:
                df = pd.read_json(self.path + f)
                if len(self.df_train) == 0:
                    self.df_train = df
                else:
                    self.df_train = self.df_train.append(df)
                print("{} Rows in Total".format(len(self.df_train)))
            except Exception as e:
                print("Got error while Reading file : " , e)
                
    @property          
    def questions(self):
        ''' 
        Returns
        ------------
        Returns English Questions in Dataframe Rows as List
        '''
        return self.df_train.question.values.tolist()
    
    @property
    def sql(self):
        '''
        Returns
        ------------
        Returns SQL in Dataframe Rows as List
        '''
        return self.df_train['query'].values.tolist()
    
    @property          
    def question_tokens(self):
        ''' 
        Returns
        ------------
        Returns English Question Tokens in Dataframe Rows as List
        '''
        
        return self.df_train['question_toks'].values.tolist()
    
    @property
    def sql_tokens(self):
        ''' 
        Returns
        ------------
        Returns SQL Query Tokens in Dataframe Rows as List
        '''
        return self.df_train['query_toks'].values.tolist()
    
    def get_special_characters(self,list_of_text):
        '''
        Parameters
        ------------
        list_of_text: Input List of Text 
        Returns
        ------------
        Provides list of Special Characters in the text
        '''
        return list(set(Preprocess.special_char(''.join([''.join(ele) for ele in list_of_text]))))
    
    def get_vocab_size(self, list_of_text):
        """
        Parameters
        ------------
        list_of_text: Input List of Text 
        
        Returns
        ------------
        Vocabulary size or unique words in the corpus
        """
        word_list = []
        for sentence in list_of_text:
            for word in sentence.split():
                word = word.lower().strip()
                if word not in word_list:
                    word_list.append(word)
        return len(word_list), word_list
        
    
    
    
    
    
    
#################################
#CONSTANTS

EOS = '[END]'
SOS = '[START]'  
    
    
    
    
    
#################################################################################################################
    
import re
import tensorflow as tf
import tensorflow_text as tf_text

class Preprocess():
    
    '''
    Preprocess class cleans and standardize the data, add SOS-EOS tags to the data
    '''
    
    def __init__(self,text):
        """
            Parameters
            ------------
            text : Input string
            Runs the text processing steps
            
        """
        self.processed_text = self.run_pipeline(text)
    
    def text_standardize(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Unicode normalization using NFKD method
            -   Lower Case text
        
        """
        text = tf_text.normalize_utf8(text, 'NFKD')
        text = tf.strings.lower(text)
        return text
    
    def text_whitespace(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Remove $ and \\ special characters 
            -   Add space around punctations
            -   Remove spaces around sentences
        
        """
        text = tf.strings.regex_replace(text, '[$\\\\]', '')
        text = tf.strings.regex_replace(text, '[.?!,¿()*:@]', r' \0 ')
        text = tf.strings.strip(text)
        return text
    
    def add_SOS_EOS(self, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Add <SOS> and <EOS> tags to each sentence
        
        """
        text = tf.strings.join([SOS, text, EOS], separator=' ')
        return text
    
    @classmethod
    def special_char(cls, text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            -   Special Characters found in Text using Regular Expression
        """
        return re.findall(r'[\W]',text.replace(' ',''))
    
    
    def run_pipeline(self,text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            Executes series of Text pre processing functions
        
        """
        text = self.text_standardize(text)
        text = self.text_whitespace(text)
        text = self.add_SOS_EOS(text)
        self.text = text
        return self.text
    
    
class Features():
    '''
    Extracts text features from data
    '''
    def tf_lower_and_split_punct(self,text):
        """
            Parameters
            ------------
            text : Input string
            
            Returns
            ------------
            Standardized Text
        
        """
        return Preprocess(text).processed_text
        
    def vectorizor(self, document, max_vocab_size):
        """
            Parameters
            ------------
            document : Collection of sentences
            max_vocab_size : No of words in document used for TextVectorization
            
            Returns
            ------------
            TextVectorization object
        
        """
        text_processor = tf.keras.layers.TextVectorization( standardize = self.tf_lower_and_split_punct, max_tokens = max_vocab_size)
        text_processor.adapt(document)
        print("Sample Vocabulary",text_processor.get_vocabulary()[:10])
        return text_processor

In [2]:
try:
    del(o)
except:
    pass
o = Train_Data('D:/DS/Learnin/Essex MS/CE888/CE888/assignment/seq2seq/spider/',['train_spider.json','train_others.json'])

o.sql[0]

Reading file at path D:/DS/Learnin/Essex MS/CE888/CE888/assignment/seq2seq/spider/train_spider.json
7000 Rows in Total
Reading file at path D:/DS/Learnin/Essex MS/CE888/CE888/assignment/seq2seq/spider/train_others.json
8659 Rows in Total


'SELECT count(*) FROM head WHERE age  >  56'

In [22]:
max_vocab_size = o.get_vocab_size(o.questions)[0]

input_text_processor = Features().vectorizor(o.questions , max_vocab_size)
output_text_processor = Features().vectorizor(o.sql , max_vocab_size)

Sample Vocabulary ['', '[UNK]', 'the', '[START]', '[END]', 'of', '?', '.', 'what', 'are']
Sample Vocabulary ['', '[UNK]', '.', 't1', 't2', '=', 'select', 'from', 'as', '[START]']


In [32]:
inp = o.questions
targ = o.sql

BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

embedding_dim = 256
units = 1024

dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [34]:
import numpy as np

embeddings_dictionary = dict()

glove_file = open(r'embeddings./glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [31]:
# from random import randint
# import numpy as np
# from numpy import array
# from numpy import argmax
# from numpy import array_equal
# from tensorflow.keras.utils import to_categorical
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import LSTM
# from keras.layers import Dense

# # generate a sequence of random integers
# def generate_sequence(length, n_unique):
# 	return [randint(1, n_unique-1) for _ in range(length)]

# # prepare data for the LSTM
# def get_dataset(n_in, n_out, cardinality, n_samples):
# 	X1, X2, y = list(), list(), list()
# 	for _ in range(n_samples):
# 		source = generate_sequence(n_in, cardinality)
#         # define padded target sequence
# 		target = source[:n_out]
# 		target.reverse()
#         # create padded input target sequence
# 		target_in = [0] + target[:-1]
#         # encode
# 		src_encoded = to_categorical([source], num_classes=cardinality)
# 		tar_encoded = to_categorical([target], num_classes=cardinality)
# 		tar2_encoded = to_categorical([target_in], num_classes=cardinality)
#         # store
# 		X1.append(src_encoded)
# 		X2.append(tar2_encoded)
# 		y.append(tar_encoded)
# 	X1 = np.squeeze(array(X1), axis=1)
# 	X2 = np.squeeze(array(X2), axis=1)
# 	y = np.squeeze(array(y), axis=1)
# 	return array(X1), array(X2), array(y)

# # returns train, inference_encoder and inference_decoder models
# def define_models(n_input, n_output, n_units):
# 	# define training encoder
# 	encoder_inputs = Input(shape=(None, n_input))
# 	encoder = LSTM(n_units, return_state=True)
# 	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 	encoder_states = [state_h, state_c]
# 	# define training decoder
# 	decoder_inputs = Input(shape=(None, n_output))
# 	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
# 	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 	decoder_dense = Dense(n_output, activation='softmax')
# 	decoder_outputs = decoder_dense(decoder_outputs)
# 	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# 	# define inference encoder
# 	encoder_model = Model(encoder_inputs, encoder_states)
# 	# define inference decoder
# 	decoder_state_input_h = Input(shape=(n_units,))
# 	decoder_state_input_c = Input(shape=(n_units,))
# 	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# 	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 	decoder_states = [state_h, state_c]
# 	decoder_outputs = decoder_dense(decoder_outputs)
# 	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# 	# return all models
# 	return model, encoder_model, decoder_model

# # generate target given source sequence
# def predict_sequence(infenc, infdec, source, n_steps, cardinality):
# 	# encode
# 	state = infenc.predict(source)
# 	# start of sequence input
# 	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
# 	# collect predictions
# 	output = list()
# 	for t in range(n_steps):
# 		# predict next char
# 		yhat, h, c = infdec.predict([target_seq] + state)
# 		# store prediction
# 		output.append(yhat[0,0,:])
# 		# update state
# 		state = [h, c]
# 		# update target sequence
# 		target_seq = yhat
# 	return array(output)

# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
# 	return [argmax(vector) for vector in encoded_seq]

# # configure problem
# n_features = 50 + 1
# n_steps_in = 6
# n_steps_out = 3
# # define model
# train, infenc, infdec = define_models(n_features, n_features, 128)
# train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# # generate training dataset
# X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
# print(X1.shape,X2.shape,y.shape)
# # train model
# train.fit([X1, X2], y, epochs=1,callbacks=[tensorboard_cb])
# # evaluate LSTM
# total, correct = 100, 0
# for _ in range(total):
# 	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
# 	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
# 	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
# 		correct += 1
# print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# # spot check some examples
# for _ in range(10):
# 	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
# 	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
# 	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)
3125/3125 [==============================] - 79s 22ms/step - loss: 0.6465 - accuracy: 0.7962
Accuracy: 99.00%
X=[47, 49, 5, 49, 38, 39] y=[5, 49, 47], yhat=[5, 49, 47]
X=[5, 20, 23, 21, 11, 39] y=[23, 20, 5], yhat=[23, 20, 5]
X=[50, 21, 46, 25, 29, 24] y=[46, 21, 50], yhat=[46, 21, 50]
X=[10, 17, 1, 5, 37, 10] y=[1, 17, 10], yhat=[1, 17, 10]
X=[31, 35, 34, 33, 12, 43] y=[34, 35, 31], yhat=[34, 35, 31]
X=[2, 43, 48, 43, 5, 18] y=[48, 43, 2], yhat=[43, 48, 2]
X=[17, 4, 2, 12, 31, 7] y=[2, 4, 17], yhat=[2, 4, 17]
X=[30, 40, 23, 36, 6, 14] y=[23, 40, 30], yhat=[23, 40, 30]
X=[34, 1, 23, 16, 18, 7] y=[23, 1, 34], yhat=[23, 1, 34]
X=[32, 23, 19, 45, 23, 37] y=[19, 23, 32], yhat=[19, 23, 32]
